Mounting to drive and importing stuff

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import torch
import torchvision
import torch.nn as nn
                                                  #  '''  do not forget =    it's (HEIGHT, WIDTH)    ''' 
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(linewidth = 120)
torch.set_grad_enabled(True)
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

from torch.utils.data.dataset import random_split

import matplotlib.pyplot as plt

import numpy as np
   
np.set_printoptions(threshold=50)    # 50 lines 


from PIL import Image 

import os

from IPython.display import clear_output
from torchvision.transforms import ToTensor

from numpy import random 

from scipy.ndimage.filters import gaussian_filter1d   ## smoother


from torch.optim.lr_scheduler import StepLR

import pickle


Load Training data

In [0]:
arr = np.array(np.load("/content/drive/My Drive/kaggle/doggonet/final_balanced_300_big.npy", allow_pickle = True))


Check whether all the labels have an equal number of instances and plot in a curve

In [0]:


doo = arr[arr[:,1].argsort()]  ## alphabetically sort wrt column 1 


print(doo[0])   # show first element 

def find_no_occ(arr, element):   ## 
    
    co = 0 
    
    for i in range(len(arr)):
        
        if arr[i][1] == element:
            co += 1
            
    return(co)



def check_for_balance(arr):
    
    dist = []
    
    for m in range(len(arr)):
        
        
        
        if arr[m][1] != arr[m-1][1]:
            
            no_occ = find_no_occ(arr, arr[m][1])
    
            dist.append(no_occ)
        
            
    plt.plot(dist)
    plt.show()
    
check_for_balance(doo)

Split loaded numpy array into columns 

In [0]:

def extract_column(arr, index):

    x_train = []
    for m in range (len(arr)):

        x_train.append(arr[m][index])

    return( np.array(x_train))

  
x_train = extract_column(arr,0)

y_train = extract_column(arr,1)
  
  

Using the `TensorDataset()` class wrapper 

And splitting the dataset into training and validation set on a 90 : 10 ratio

In [0]:


x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()



full_dataset = TensorDataset(x_train_tensor, y_train_tensor)       ## make compatible with DataLoader 


train_size = int(0.9 * len(full_dataset))  ## 80/20 split
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])



Setting batch sizes of 120 and 12 for train and test set, the train set has 120 possible labels which are randomly shuffled, hence the batch size of 120 would (approximately) contain all the labels in a balanced manner 


In [0]:
## using a dataloader 

train_loader = DataLoader(dataset=train_dataset, batch_size=120, shuffle=True)

val_loader = DataLoader(dataset=test_dataset, batch_size=12, drop_last = True)


print(len(train_loader))

Some simple functions which come in handy when visualising data 

In [0]:

    

## test NN output

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()





def vis_loss_multi(loss_arr,val_arr):
  
    plt.plot(loss_arr)
    plt.plot(val_arr)
    plt.show()


*Doggonet needs no introduction*

4 2D convolutional layers and 3 linear layers with all the bells and whistles included in between 

In [0]:


## doggonet 


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        
        
        self.conv1 = nn.Conv2d(3, 30, 5)           
        
        self.pool = nn.MaxPool2d(3,3)
        
        self.dropout = nn.Dropout(p=0.5)
        
        self.conv2 = nn.Conv2d(30,30,5)
        self.conv3 = nn.Conv2d(30,30,5)
        
        self.conv3_bn = nn.BatchNorm2d(30)    
        
        self.conv4 = nn.Conv2d(30,30,5)
        

        
        self.fc1 = nn.Linear(4320, 900) 
        
        self.dropout = nn.Dropout(p=0.5)  # drop
        
        self.fc2 = nn.Linear(900, 900)
        self.fc3 = nn.Linear(900, 360)
        
        
        
        self.conv3_bn_linear = nn.BatchNorm1d(360)    
     
        
        
        self.fc4 = nn.Linear(360, 120)   ## output 
  

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
                                                                        ##   F.relu
        x = self.pool(x)
     
        x = self.dropout(x)
        
        x = F.relu(self.conv2(x))
        
        x = self.pool(x)
        
        x = (self.conv3(x))
        
        x = F.relu(self.conv3_bn(x))
    
        
        x = F.relu(self.conv4(x))
                             
        x = x.view(x.size(0), -1)
               
        
        x = F.relu(self.fc1(x))
        
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)
        
        x = F.relu(x)
        
        x = self.fc4(x)
 
        
        
        
        return x

doggonet = Net()

print(doggonet)

`make_train_step():` a single batch passes through CNN and backpropagation occurs 
    
    
`train():` basically trains the NN for n_epochs 





In [0]:


def make_train_step(model, optimizer):

    def train_step(x, y):

        pred = model(x)
        
        
        y = y.long()
        
        
        loss = F.cross_entropy(pred, y)
        

        
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()
        
        print ("|", end = "")
        return loss.item()
      
    return train_step


model = doggonet

learning_rate = 0.091
 
optimizer = optim.SGD(doggonet.parameters(), learning_rate)                             #,momentum = 0.5, nesterov = True)  ## nesterov is true 

train_step = make_train_step(model, optimizer)
scheduler = StepLR(optimizer, step_size=2, gamma=0.9)  # lr decreases by 10% after every 2 epochs


losses = []
val_losses = []



def train(train_loader, epochs):
  

  for i in range(epochs):
    
    print("epoch = ", i+1)
    
    foo = optimizer.param_groups[0]['lr']


    
    print ("current lr = ", foo)
    print("                    |                                                                                                                                                        |")   
    print("training batches  --", end = "")

   
    for x_batch, y_batch in train_loader:
      
        model.train()


        loss = train_step(x_batch, y_batch)

        losses.append(loss)
      
    with torch.no_grad():
        for x_val, y_val in val_loader:


            model.eval()

            y_val = y_val.long()

            yhat = doggonet(x_val)  # pred 
            val_loss = F.cross_entropy(yhat, y_val)
            val_losses.append(val_loss.item())


    
    scheduler.step()   ## lr decay caller 
      
    
    batch_avg_loss = sum(losses[-120:])/120
    print ("")
    
    print("epoch avg loss = ", batch_avg_loss)

    print (" on the validadtion set -- global")
    
    validation_avg_loss = (sum(val_losses[-12:])/12)
    
    print (" validation set avg loss in last epoch = ", validation_avg_loss)
    
    vis_loss_multi(losses,val_losses)
    
    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    
    
    print("")

    print ("training complete -------------------------------------------")


In [0]:
train(train_loader, 10) 


Save the trained nn

In [0]:
nn_name = "/content/drive/My Drive/kaggle/doggonet/models/" + "10_epochs_doggonet_batchwise.pth"

torch.save(doggonet, nn_name)
